# Module 09: Preregistration & Open Science

**Estimated Time:** 45 minutes

## Learning Objectives

By the end of this module, you will be able to:

1. Explain the rationale for preregistration and open science practices
2. Identify questionable research practices (QRPs) that preregistration prevents
3. Create a preregistration for a research study
4. Use the Open Science Framework (OSF) for project management
5. Understand registered reports and their benefits
6. Apply FAIR principles to research data and materials
7. Navigate the reproducibility crisis and solutions
8. Handle deviations from preregistered plans appropriately

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings("ignore")

# Set style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["font.size"] = 11

# Create output directory
import os

os.makedirs("../notebooks/outputs/module_09", exist_ok=True)

print("✓ Libraries imported successfully")
print("✓ Output directory created")

## 1. Why Preregistration Matters

### The Problem: Questionable Research Practices (QRPs)

**Preregistration** is the practice of publicly documenting research plans (hypotheses, methods, analyses) before data collection begins.

#### Common QRPs that preregistration prevents:

1. **P-hacking** (Data dredging): Testing multiple analyses and reporting only significant results
2. **HARKing** (Hypothesizing After Results are Known): Presenting exploratory findings as confirmatory
3. **Selective reporting**: Omitting non-significant results or conditions
4. **Flexible stopping**: Collecting data until significance is reached
5. **Optional outlier exclusion**: Removing data points to achieve significance

Let's demonstrate the danger of p-hacking:

In [ ]:
def simulate_p_hacking(n_per_group=30, n_simulations=1000, seed=42):
    """
    Simulate p-hacking by testing multiple dependent variables.

    We have NO real effect, but test 10 DVs and report the 'best' one.
    """
    np.random.seed(seed)

    # Track false positive rates
    honest_false_positives = 0  # Testing only DV1
    phacked_false_positives = 0  # Testing all 10 DVs, reporting best p-value

    for sim in range(n_simulations):
        # Two groups with NO real difference (null hypothesis is true)
        group1_dvs = np.random.normal(0, 1, (n_per_group, 10))  # 10 DVs
        group2_dvs = np.random.normal(0, 1, (n_per_group, 10))  # 10 DVs

        # Honest researcher: test only DV1
        t_stat, p_value = stats.ttest_ind(group1_dvs[:, 0], group2_dvs[:, 0])
        if p_value < 0.05:
            honest_false_positives += 1

        # P-hacker: test all 10 DVs, report the smallest p-value
        p_values = []
        for dv in range(10):
            t_stat, p_value = stats.ttest_ind(group1_dvs[:, dv], group2_dvs[:, dv])
            p_values.append(p_value)

        min_p = min(p_values)
        if min_p < 0.05:
            phacked_false_positives += 1

    honest_rate = honest_false_positives / n_simulations * 100
    phacked_rate = phacked_false_positives / n_simulations * 100

    return honest_rate, phacked_rate


# Run simulation
honest_rate, phacked_rate = simulate_p_hacking()

print("🔬 P-HACKING SIMULATION")
print("=" * 50)
print(f"Scenario: Two groups with NO real difference (H₀ is true)")
print(f"Sample size: 30 per group")
print(f"Number of simulations: 1,000\n")

print(f"Honest researcher (tests only DV1):")
print(f"  False positive rate: {honest_rate:.1f}%")
print(f"  ✓ Close to nominal 5% Type I error rate\n")

print(f"P-hacker (tests 10 DVs, reports 'best' one):")
print(f"  False positive rate: {phacked_rate:.1f}%")
print(f"  ✗ Inflated by {phacked_rate/honest_rate:.1f}x!\n")

print("⚠️  P-hacking dramatically increases false discoveries!")
print("✓  Preregistration prevents this by committing to analyses upfront.")

In [ ]:
# Visualize the danger of multiple testing
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left plot: False positive rate by number of tests
n_tests = np.arange(1, 21)
false_positive_rate = 1 - (1 - 0.05) ** n_tests

axes[0].plot(n_tests, false_positive_rate * 100, "o-", linewidth=2, markersize=8, color="#e74c3c")
axes[0].axhline(y=5, color="green", linestyle="--", linewidth=2, label="Nominal 5%", alpha=0.7)
axes[0].set_xlabel("Number of Tests Conducted", fontsize=12, fontweight="bold")
axes[0].set_ylabel(
    "Probability of At Least One\nFalse Positive (%)", fontsize=12, fontweight="bold"
)
axes[0].set_title("The Multiple Testing Problem", fontsize=14, fontweight="bold")
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0, 70])

# Add annotations
axes[0].annotate(
    "With 10 tests,\n40% chance of\nfalse positive!",
    xy=(10, false_positive_rate[9] * 100),
    xytext=(12, 20),
    arrowprops=dict(arrowstyle="->", color="red", lw=2),
    fontsize=10,
    color="red",
    fontweight="bold",
)

# Right plot: Research practices comparison
practices = [
    "Single\npreregistered\ntest",
    "Multiple tests\n(corrected)",
    "P-hacking\n(uncorrected)",
]
rates = [5, 5, 40]  # Approximate rates
colors = ["#27ae60", "#f39c12", "#e74c3c"]

bars = axes[1].bar(practices, rates, color=colors, alpha=0.7, edgecolor="black", linewidth=2)
axes[1].axhline(y=5, color="green", linestyle="--", linewidth=2, alpha=0.7)
axes[1].set_ylabel("False Positive Rate (%)", fontsize=12, fontweight="bold")
axes[1].set_title("Research Practices Comparison", fontsize=14, fontweight="bold")
axes[1].set_ylim([0, 50])

# Add value labels on bars
for bar, rate in zip(bars, rates):
    height = bar.get_height()
    axes[1].text(
        bar.get_x() + bar.get_width() / 2.0,
        height + 1,
        f"{rate}%",
        ha="center",
        va="bottom",
        fontsize=12,
        fontweight="bold",
    )

plt.tight_layout()
plt.savefig(
    "../notebooks/outputs/module_09/p_hacking_demonstration.png", dpi=300, bbox_inches="tight"
)
plt.show()

print("✓ Figure saved to outputs/module_09/p_hacking_demonstration.png")

### Key Insight

Even with NO real effect, testing 10 dependent variables gives you a ~40% chance of finding at least one "significant" result!

**Preregistration solution:** Commit to which DVs you'll test BEFORE seeing the data.

## 2. Open Science Framework (OSF)

The **Open Science Framework** (https://osf.io) is a free platform for:
- Project management
- Preregistration
- Data/materials sharing
- Collaboration
- DOI generation

### OSF Preregistration Workflow

```
1. Create OSF Project
   ↓
2. Choose Preregistration Template
   ↓
3. Complete Registration Form
   - Research questions
   - Hypotheses
   - Design and methods
   - Analysis plan
   - Sample size justification
   ↓
4. Submit Registration (creates timestamped, uneditable copy)
   ↓
5. Embargo period (optional, 0-4 years)
   ↓
6. Conduct study following preregistered plan
   ↓
7. Report deviations transparently in manuscript
   ↓
8. Make registration public with publication
```

### Popular Preregistration Templates

Let's create a simple preregistration template:

In [ ]:
def create_preregistration_template(study_type="experiment"):
    """
    Generate a basic preregistration template.

    Parameters:
    -----------
    study_type : str
        Type of study ('experiment', 'observational', 'meta_analysis')
    """
    template = f"""
PREREGISTRATION TEMPLATE ({study_type.upper()})
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
{'=' * 70}

1. STUDY INFORMATION
   Title: [Your study title]
   Authors: [List all authors]
   Research Questions: [Main questions to be addressed]

2. HYPOTHESES
   H1: [First hypothesis]
   H2: [Second hypothesis]
   ...
   
   Direction: [Directional (one-tailed) or non-directional (two-tailed)]

3. DESIGN
   Study Type: {study_type}
   Design: [e.g., 2x2 between-subjects factorial]
   Independent Variables: [List IVs and levels]
   Dependent Variables: [List DVs and measurement methods]
   
4. SAMPLING PLAN
   Target Population: [Define population]
   Inclusion Criteria: [Who can participate]
   Exclusion Criteria: [Who cannot participate]
   Sample Size: [N = XXX]
   Power Analysis: [Describe justification for sample size]
   Stopping Rule: [When will data collection end?]

5. PROCEDURE
   [Detailed description of study procedures]
   [Step-by-step participant experience]

6. ANALYSIS PLAN
   Primary Analysis:
     - Statistical test: [e.g., independent samples t-test]
     - Alpha level: [typically 0.05]
     - Variables: [DV predicted by IV]
   
   Secondary Analyses:
     - [List any planned secondary analyses]
   
   Inference Criteria:
     - What will you conclude if H1 is supported?
     - What will you conclude if H1 is not supported?

7. DATA EXCLUSION
   Outlier Definition: [How will outliers be identified?]
   Outlier Treatment: [Will they be excluded or retained?]
   Missing Data: [How will missing data be handled?]
   Attention Checks: [Any quality control measures?]

8. VARIABLES
   Manipulated Variables:
     - [List each variable and how it's operationalized]
   
   Measured Variables:
     - [List each variable and measurement instrument]

9. ADDITIONAL INFORMATION
   Prior Studies: [Have you conducted pilot studies?]
   Data Collection: [When will data be collected?]
   Funding: [Any funding sources?]
   Conflicts of Interest: [Declare any COIs]

{'=' * 70}
NOTE: This is a simplified template. Use OSF, AsPredicted, or journal-
specific templates for actual preregistrations.
"""
    return template


# Generate example template
prereg = create_preregistration_template("experiment")
print(prereg)

# Save template
with open("../notebooks/outputs/module_09/preregistration_template.txt", "w") as f:
    f.write(prereg)

print("\n✓ Template saved to outputs/module_09/preregistration_template.txt")

### Example: Completed Preregistration

Let's fill out a simple preregistration for a hypothetical study:

In [ ]:
# Example completed preregistration
example_prereg = """
PREREGISTRATION: EFFECT OF STUDY ENVIRONMENT ON EXAM PERFORMANCE
===============================================================================
Date: 2025-01-15
Platform: OSF (osf.io/xxxxx)

1. STUDY INFORMATION
   Title: Does Background Music Affect Exam Performance in Undergraduates?
   Authors: Smith, J. & Jones, A.
   Research Question: Does listening to background music while studying 
                      affect exam performance compared to silence?

2. HYPOTHESES
   H1: Students who study with background music will score LOWER on exams
       than students who study in silence.
   
   Rationale: Cognitive load theory suggests background music creates 
             interference during encoding.
   
   Direction: One-tailed (directional)

3. DESIGN
   Study Type: Randomized Controlled Trial (RCT)
   Design: Between-subjects, two conditions
   
   IV: Study environment (2 levels)
       - Music condition: Instrumental music at 60 dB
       - Silence condition: <30 dB background noise
   
   DV: Exam score (0-100 points)
       - 20-question multiple-choice test on assigned chapter
       - Validated test with α = 0.82 reliability

4. SAMPLING PLAN
   Population: Undergraduate psychology students at State University
   
   Inclusion: Age 18-25, native English speakers, normal hearing
   Exclusion: Diagnosed learning disabilities, prior course in topic
   
   Sample Size: N = 128 (64 per condition)
   
   Power Analysis:
     - Expected effect: d = 0.5 (medium, based on Smith et al., 2023)
     - Power: 0.80
     - Alpha: 0.05 (one-tailed)
     - Required N: 128
   
   Stopping Rule: Data collection ends when N = 128, or May 1, 2025,
                  whichever comes first.

5. PROCEDURE
   1. Participants arrive at lab, complete consent form
   2. Random assignment to music or silence condition (blocked randomization)
   3. 30-minute study period with assigned textbook chapter
   4. 10-minute break (no study materials)
   5. 20-minute exam on chapter content
   6. Debrief and compensation ($15)

6. ANALYSIS PLAN
   Primary Analysis:
     - Test: Independent samples t-test (one-tailed)
     - Alpha: 0.05
     - Variables: Exam score ~ Study condition
     - Software: Python (SciPy.stats)
   
   Effect Size: Cohen's d with 95% CI
   
   Inference:
     - If p < 0.05 AND Music < Silence: H1 supported
     - If p ≥ 0.05 OR Music ≥ Silence: H1 not supported
   
   No secondary analyses planned.

7. DATA EXCLUSION
   Outliers: Scores > 3 SD from condition mean will be flagged but RETAINED
   Missing Data: Participants who don't complete exam will be excluded from
                 analysis (but reported in CONSORT diagram)
   Quality Check: Post-study questionnaire asks about distractions;
                  major incidents (e.g., fire alarm) → exclusion

8. EXPLORATORY ANALYSES (NOT CONFIRMATORY)
   - We may explore moderation by prior GPA
   - We may explore correlation between study time and exam score
   - These will be clearly labeled as exploratory in manuscript

===============================================================================
TIMESTAMP: This preregistration was submitted to OSF on 2025-01-15 at 14:32 UTC
and cannot be edited. Data collection begins 2025-01-20.
"""

print(example_prereg)

# Save example
with open("../notebooks/outputs/module_09/example_preregistration.txt", "w") as f:
    f.write(example_prereg)

print("\n✓ Example saved to outputs/module_09/example_preregistration.txt")

## 3. Registered Reports

**Registered Reports** are a publication format where:
1. **Stage 1:** Submit introduction + methods for peer review BEFORE data collection
2. **In-principle acceptance:** If methods are sound, journal commits to publish regardless of results
3. **Stage 2:** After data collection, submit full paper for review

### Benefits
- ✓ Eliminates publication bias against null results
- ✓ Reviews focus on methods quality, not "exciting" results
- ✓ Prevents p-hacking and HARKing
- ✓ Reduces incentive to oversell findings

### Timeline Comparison

In [ ]:
# Visualize registered report timeline
fig, ax = plt.subplots(figsize=(14, 8))

# Traditional publishing
trad_stages = [
    ("Design Study", 0, 2),
    ("Collect Data", 2, 6),
    ("Analyze Data", 6, 8),
    ("Write Paper", 8, 10),
    ("Submit", 10, 10.5),
    ("Peer Review", 10.5, 14),
    ("Revisions", 14, 16),
    ("Accept/Publish", 16, 17),
]

# Registered report
rr_stages = [
    ("Design Study", 0, 2),
    ("Write Stage 1", 2, 3),
    ("Submit Stage 1", 3, 3.5),
    ("Stage 1 Review", 3.5, 6),
    ("Revise Stage 1", 6, 7),
    ("In-Principle\nAcceptance", 7, 7.5),
    ("Collect Data", 7.5, 11.5),
    ("Analyze Data", 11.5, 13.5),
    ("Write Stage 2", 13.5, 15),
    ("Submit Stage 2", 15, 15.5),
    ("Stage 2 Review", 15.5, 17),
    ("Final Publish", 17, 18),
]

# Plot traditional
y_pos = 2
for stage, start, end in trad_stages:
    color = "#3498db" if "Review" in stage or "Revisions" in stage else "#95a5a6"
    if "Collect" in stage or "Analyze" in stage:
        color = "#e74c3c"
    ax.barh(
        y_pos,
        end - start,
        left=start,
        height=0.6,
        color=color,
        edgecolor="black",
        linewidth=1.5,
        alpha=0.8,
    )
    ax.text(
        (start + end) / 2,
        y_pos,
        stage,
        ha="center",
        va="center",
        fontsize=9,
        fontweight="bold",
        color="white",
    )

# Plot registered report
y_pos = 1
for stage, start, end in rr_stages:
    color = "#3498db" if "Review" in stage else "#95a5a6"
    if "Collect" in stage or "Analyze" in stage:
        color = "#e74c3c"
    if "Acceptance" in stage:
        color = "#27ae60"
    ax.barh(
        y_pos,
        end - start,
        left=start,
        height=0.6,
        color=color,
        edgecolor="black",
        linewidth=1.5,
        alpha=0.8,
    )
    ax.text(
        (start + end) / 2,
        y_pos,
        stage,
        ha="center",
        va="center",
        fontsize=8,
        fontweight="bold",
        color="white",
    )

# Formatting
ax.set_yticks([1, 2])
ax.set_yticklabels(["Registered Report", "Traditional Publishing"], fontsize=12, fontweight="bold")
ax.set_xlabel("Months from Start", fontsize=12, fontweight="bold")
ax.set_title("Publication Timeline Comparison", fontsize=14, fontweight="bold", pad=20)
ax.set_xlim([0, 18])
ax.grid(axis="x", alpha=0.3, linestyle="--")

# Add legend
from matplotlib.patches import Patch

legend_elements = [
    Patch(facecolor="#e74c3c", edgecolor="black", label="Research Activities"),
    Patch(facecolor="#3498db", edgecolor="black", label="Peer Review"),
    Patch(facecolor="#27ae60", edgecolor="black", label="In-Principle Acceptance"),
    Patch(facecolor="#95a5a6", edgecolor="black", label="Other"),
]
ax.legend(handles=legend_elements, loc="upper right", fontsize=10)

# Add annotation
ax.annotate(
    "Key difference:\nMethods reviewed\nBEFORE data collection",
    xy=(5, 1),
    xytext=(8, 0.3),
    arrowprops=dict(arrowstyle="->", color="#27ae60", lw=3),
    fontsize=11,
    color="#27ae60",
    fontweight="bold",
    bbox=dict(boxstyle="round,pad=0.5", facecolor="white", edgecolor="#27ae60", linewidth=2),
)

plt.tight_layout()
plt.savefig(
    "../notebooks/outputs/module_09/registered_report_timeline.png", dpi=300, bbox_inches="tight"
)
plt.show()

print("✓ Timeline comparison saved")

### Journals Offering Registered Reports

Over 300 journals now offer Registered Reports format:
- *Psychological Science*
- *Nature Human Behaviour*
- *Royal Society Open Science*
- *Cortex*
- Many more...

See complete list: https://www.cos.io/initiatives/registered-reports

## 4. PROSPERO for Systematic Reviews

**PROSPERO** (https://www.crd.york.ac.uk/prospero/) is the international database for preregistering systematic reviews.

### Why Preregister Systematic Reviews?

1. Prevents selective reporting of outcomes
2. Reduces duplication (you can see if someone already registered similar review)
3. Increases transparency
4. Required by Cochrane and many journals

### What to Preregister:

- Research question (PICO format)
- Inclusion/exclusion criteria
- Search strategy and databases
- Screening process
- Risk of bias assessment tools
- Data extraction plan
- Synthesis methods (meta-analysis, narrative, etc.)

## 5. FAIR Principles for Open Data

The **FAIR** principles guide open data practices:

### F - Findable
- Assign persistent identifier (DOI)
- Rich metadata
- Indexed in searchable resource

### A - Accessible
- Retrievable by identifier
- Open, free protocol
- Metadata remains even if data is removed

### I - Interoperable
- Use standard formats (CSV, not proprietary)
- Controlled vocabularies
- Include links to related data

### R - Reusable
- Clear usage license
- Detailed provenance
- Meets community standards

### Data Sharing Platforms

In [ ]:
# Create comparison of data sharing platforms
platforms_data = {
    "Platform": ["OSF", "Zenodo", "Figshare", "Dryad", "GitHub", "Dataverse"],
    "Free": ["Yes", "Yes", "Yes", "Fee > 50GB", "Yes", "Yes"],
    "DOI": ["Yes", "Yes", "Yes", "Yes", "No", "Yes"],
    "Size Limit": ["5 GB/file", "50 GB", "20 GB", "Unlimited", "100 MB/file", "Unlimited"],
    "Versioning": ["Yes", "Yes", "Yes", "Yes", "Yes", "Yes"],
    "Best For": ["Psychology", "General", "General", "Ecology", "Code", "Social Science"],
}

platforms_df = pd.DataFrame(platforms_data)
print("📊 DATA SHARING PLATFORMS COMPARISON")
print("=" * 80)
print(platforms_df.to_string(index=False))
print("\n💡 TIP: Choose platform based on your field's norms and data size.")

# Save table
platforms_df.to_csv("../notebooks/outputs/module_09/data_sharing_platforms.csv", index=False)
print("\n✓ Table saved to outputs/module_09/data_sharing_platforms.csv")

### Choosing a License

**Creative Commons Licenses** for data/materials:

- **CC0**: Public domain, no restrictions (most open)
- **CC-BY**: Requires attribution
- **CC-BY-SA**: Attribution + share-alike
- **CC-BY-NC**: Attribution + non-commercial only

**Code Licenses:**

- **MIT**: Very permissive
- **GPL**: Requires derivative works to also be open
- **Apache 2.0**: Permissive with patent protection

**Recommendation:** For maximum reusability, use CC0 or CC-BY for data, and MIT or Apache for code.

## 6. The Reproducibility Crisis

### Key Findings

**Replication Failures:**
- Psychology: 36% of studies replicated (Open Science Collaboration, 2015)
- Biomedical: 11% of landmark studies replicated (Begley & Ellis, 2012)
- Economics: 61% of studies replicated (Camerer et al., 2016)

### Contributing Factors

In [ ]:
# Visualize factors contributing to reproducibility crisis
factors = [
    "Publication\nBias",
    "P-Hacking",
    "HARKing",
    "Underpowered\nStudies",
    "Poor\nDocumentation",
    "Data/Code\nNot Shared",
    "Lack of\nReplication",
    "Flexible\nAnalyses",
]

# Estimated contribution (illustrative)
contributions = [25, 20, 15, 12, 10, 8, 6, 4]

# Solutions for each
solutions = [
    "Registered Reports",
    "Preregistration",
    "Preregistration",
    "Power Analysis",
    "Open Methods",
    "Open Data/Code",
    "Replication Studies",
    "Preregistration",
]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Left: Pie chart of factors
colors = plt.cm.Reds(np.linspace(0.4, 0.9, len(factors)))
wedges, texts, autotexts = ax1.pie(
    contributions,
    labels=factors,
    autopct="%1.0f%%",
    colors=colors,
    startangle=90,
    textprops={"fontsize": 10, "fontweight": "bold"},
)
ax1.set_title(
    "Factors Contributing to\nReproducibility Crisis", fontsize=14, fontweight="bold", pad=20
)

# Right: Solutions bar chart
solution_counts = pd.Series(solutions).value_counts()
colors_solutions = ["#27ae60", "#3498db", "#e67e22", "#9b59b6", "#e74c3c"]

bars = ax2.barh(
    solution_counts.index,
    solution_counts.values,
    color=colors_solutions[: len(solution_counts)],
    edgecolor="black",
    linewidth=2,
    alpha=0.8,
)
ax2.set_xlabel("Number of Problems Addressed", fontsize=12, fontweight="bold")
ax2.set_title("Open Science Solutions", fontsize=14, fontweight="bold", pad=20)
ax2.grid(axis="x", alpha=0.3, linestyle="--")

# Add value labels
for bar in bars:
    width = bar.get_width()
    ax2.text(
        width + 0.1,
        bar.get_y() + bar.get_height() / 2,
        f"{int(width)}",
        ha="left",
        va="center",
        fontsize=11,
        fontweight="bold",
    )

plt.tight_layout()
plt.savefig(
    "../notebooks/outputs/module_09/reproducibility_crisis.png", dpi=300, bbox_inches="tight"
)
plt.show()

print("✓ Reproducibility crisis visualization saved")

## 7. Handling Deviations from Preregistration

**Reality check:** You will probably need to deviate from your preregistration!

### Common Reasons for Deviation

1. Data collection issues (e.g., lower recruitment than expected)
2. Assumption violations (e.g., non-normality requires different test)
3. Errors in preregistration (e.g., typos, unclear wording)
4. Unforeseen circumstances (e.g., COVID-19 pandemic)

### Best Practices

In [ ]:
deviation_guide = """
HANDLING DEVIATIONS FROM PREREGISTRATION: DECISION TREE
========================================================================

Did you deviate from the preregistration?
│
├─ NO → Great! Report: "All analyses followed preregistered plan."
│
└─ YES → Continue below...
    │
    └─ When did you decide to deviate?
        │
        ├─ BEFORE seeing the data (e.g., during data collection)
        │   └─ Action: Create a dated addendum to preregistration
        │       - Explain reason for change
        │       - Post to OSF with timestamp
        │       - Report in paper: "We deviated from our preregistered
        │         plan before data analysis (see dated addendum at
        │         osf.io/xxxxx). Specifically, we [describe change]."
        │
        └─ AFTER seeing the data
            └─ Was deviation necessary or optional?
                │
                ├─ NECESSARY (e.g., assumption violation)
                │   └─ Action: Report transparently
                │       - Explain why change was necessary
                │       - Describe original plan and deviation
                │       - If possible, report BOTH analyses
                │       - Report in paper: "Due to [reason], we
                │         necessarily deviated from our preregistered
                │         analysis plan. Originally we planned [X],
                │         but [reason] required us to use [Y] instead."
                │
                └─ OPTIONAL (e.g., exploratory analysis)
                    └─ Action: Label as EXPLORATORY
                        - Clearly distinguish confirmatory from exploratory
                        - Do NOT present exploratory as confirmatory
                        - Report in paper: "In addition to our
                          preregistered confirmatory analyses, we
                          conducted the following EXPLORATORY analyses..."
                        - Interpret with appropriate caution

========================================================================

KEY PRINCIPLES:
✓ TRANSPARENCY: Always report deviations clearly
✓ HONESTY: Don't hide or downplay changes
✓ DISTINCTION: Separate confirmatory from exploratory
✓ JUSTIFICATION: Explain why deviation was necessary

❌ NEVER:
   - Pretend deviation didn't happen
   - Present exploratory findings as confirmatory
   - Change preregistration after seeing data

✓ REMEMBER: Deviations don't invalidate preregistration!
           The goal is transparency, not perfection.
"""

print(deviation_guide)

# Save guide
with open("../notebooks/outputs/module_09/deviation_guide.txt", "w") as f:
    f.write(deviation_guide)

print("\n✓ Deviation guide saved to outputs/module_09/deviation_guide.txt")

### Example Deviation Reporting

**Good example from a published paper:**

> *"We preregistered our hypotheses and analysis plan (osf.io/abc123). We deviated from our preregistration in two ways. First, we originally planned to use ANOVA but discovered severe violations of homogeneity of variance (Levene's test, p < .001), so we used Welch's ANOVA instead. Second, we preregistered N = 200 but only recruited N = 156 due to pandemic-related disruptions. Our sensitivity analysis indicated we still had 80% power to detect d = 0.46, slightly larger than our anticipated d = 0.40. In addition to our confirmatory analyses, we report exploratory analyses of potential moderators in Appendix B."*

This demonstrates:
- ✓ Clear statement of deviations
- ✓ Justification for each change
- ✓ Impact assessment (power analysis)
- ✓ Distinction between confirmatory and exploratory

## 8. Practical Checklist

### Before Data Collection

In [ ]:
# Create comprehensive preregistration checklist
checklist = """
PREREGISTRATION CHECKLIST
==================================================
Use this before submitting your preregistration.

STUDY DESIGN
☐ Research question clearly stated
☐ Hypotheses are specific and testable
☐ Direction specified (one-tailed vs. two-tailed)
☐ Independent and dependent variables operationalized
☐ Study design justified (RCT, quasi-experimental, etc.)

SAMPLING
☐ Target population defined
☐ Inclusion/exclusion criteria specified
☐ Sample size justified with power analysis
☐ Stopping rule stated (when does data collection end?)
☐ Recruitment strategy described

PROCEDURE
☐ Step-by-step procedure documented
☐ Randomization method specified (if applicable)
☐ Blinding procedures described (if applicable)
☐ Materials shared or described in detail

ANALYSIS PLAN
☐ Primary statistical test(s) specified
☐ Alpha level stated (typically 0.05)
☐ Primary dependent variable(s) identified
☐ Covariates listed (if any)
☐ Inference criteria clear (what constitutes support for H1?)
☐ Secondary analyses described (if any)
☐ Exploratory analyses distinguished from confirmatory

DATA QUALITY
☐ Outlier definition and treatment specified
☐ Missing data plan described
☐ Attention check criteria stated (if applicable)
☐ Data exclusion rules predefined

DOCUMENTATION
☐ Preregistration submitted to public registry (OSF, AsPredicted, PROSPERO)
☐ Timestamp/DOI obtained
☐ All collaborators have reviewed preregistration
☐ Preregistration will be made public (set embargo if needed)

AFTER DATA COLLECTION
☐ Compare actual procedures to preregistration
☐ Document any deviations with justification
☐ Report confirmatory vs. exploratory analyses separately
☐ Link to preregistration in manuscript
☐ Share data and code (if possible)

==================================================
💡 TIP: It's better to have an imperfect preregistration
        than none at all. You can always report deviations!
"""

print(checklist)

# Save checklist
with open("../notebooks/outputs/module_09/preregistration_checklist.txt", "w") as f:
    f.write(checklist)

print("\n✓ Checklist saved to outputs/module_09/preregistration_checklist.txt")

## 9. Practice Exercise

### Scenario: Sleep and Memory Study

You're planning a study to test whether sleep quality affects memory consolidation. Create a preregistration addressing:

1. **Research Question:** Does sleep quality predict memory consolidation?

2. **Design:** Observational study
   - Participants complete memory task before bed
   - Sleep quality measured with actigraphy
   - Memory test repeated next morning

3. **Your Task:** Complete this preregistration template

In [ ]:
exercise_template = """
PRACTICE PREREGISTRATION: SLEEP AND MEMORY
========================================================================

INSTRUCTIONS: Fill in each section with your planned details.

1. HYPOTHESES
   H1: [Your directional hypothesis about sleep quality and memory]
   
   Direction: [One-tailed or two-tailed? Why?]

2. VARIABLES
   Primary IV: [How will you measure sleep quality?]
               [What metric? (e.g., sleep efficiency, total sleep time)]
   
   Primary DV: [How will you measure memory consolidation?]
               [e.g., % retention from night before to morning]
   
   Covariates: [What variables might you need to control for?]
               [e.g., age, baseline memory, caffeine consumption]

3. SAMPLE
   N = [How many participants?]
   
   Power justification:
   [Use a power analysis to justify your N]
   [What effect size are you powered to detect?]
   
   Inclusion: [Who can participate?]
   Exclusion: [Who cannot? (e.g., sleep disorders, medications)]

4. ANALYSIS PLAN
   Primary test: [What statistical test? (e.g., correlation, regression)]
   
   Model: [If regression, what will your model be?]
          [e.g., Memory retention ~ Sleep quality + Age + Baseline memory]
   
   Alpha: [Significance level]
   
   Inference: [When will you conclude H1 is supported?]

5. DATA QUALITY
   Outliers: [How will you identify outliers?]
             [Will you exclude or retain them?]
   
   Missing data: [What if actigraphy fails for some participants?]
   
   Quality checks: [Any attention checks or manipulation checks?]

========================================================================

REFLECTION QUESTIONS:
1. What are the weaknesses of your design?
2. What alternative explanations exist for your predicted results?
3. What would you do if your hypothesis is NOT supported?
4. Are there any "researcher degrees of freedom" you need to constrain?

"""

print(exercise_template)

# Save exercise
with open("../notebooks/outputs/module_09/practice_preregistration.txt", "w") as f:
    f.write(exercise_template)

print("\n✓ Practice exercise saved to outputs/module_09/practice_preregistration.txt")
print("\n💡 Take 15-20 minutes to complete this exercise on your own.")
print("   Compare your answers with published preregistrations on OSF.")

## 10. Summary

### Key Takeaways

1. **Preregistration prevents QRPs**
   - P-hacking and HARKing inflate false positive rates
   - Committing to analyses upfront maintains α = 0.05

2. **Multiple platforms available**
   - OSF: General purpose, flexible
   - AsPredicted: Quick, streamlined
   - PROSPERO: Systematic reviews
   - ClinicalTrials.gov: Clinical trials

3. **Registered Reports eliminate publication bias**
   - Stage 1 review before data collection
   - In-principle acceptance regardless of results
   - Growing adoption across journals

4. **FAIR principles guide open data**
   - Findable: DOIs and metadata
   - Accessible: Open repositories
   - Interoperable: Standard formats
   - Reusable: Clear licenses

5. **Deviations are okay if reported transparently**
   - Document reasons for changes
   - Distinguish confirmatory from exploratory
   - The goal is transparency, not perfection

### Best Practices

✓ **Do:**
- Preregister before looking at the data
- Be as specific as possible
- Report all deviations transparently
- Share data and materials when possible
- Distinguish confirmatory from exploratory analyses

✗ **Don't:**
- Edit preregistration after seeing data
- Present exploratory findings as confirmatory
- Hide deviations or failed studies
- Let perfect be the enemy of good

### Impact

Open science practices:
- Improve credibility of findings
- Accelerate scientific progress
- Increase public trust in science
- Reduce research waste
- Make science more equitable and accessible

## Additional Resources

### Platforms
- Open Science Framework: https://osf.io
- AsPredicted: https://aspredicted.org
- PROSPERO: https://www.crd.york.ac.uk/prospero/
- ClinicalTrials.gov: https://clinicaltrials.gov
- Registered Reports: https://www.cos.io/initiatives/registered-reports

### Readings
- Nosek, B. A., et al. (2018). The preregistration revolution. *PNAS, 115*(11), 2600-2606.
- Simmons, J. P., et al. (2011). False-positive psychology. *Psychological Science, 22*(11), 1359-1366.
- Chambers, C. (2017). *The Seven Deadly Sins of Psychology*. Princeton University Press.
- Wilkinson, M. D., et al. (2016). The FAIR Guiding Principles. *Scientific Data, 3*, 160018.
- Open Science Collaboration (2015). Estimating the reproducibility of psychological science. *Science, 349*(6251), aac4716.

### Tutorials
- OSF Preregistration Guide: https://help.osf.io/hc/en-us/articles/360019738834
- APA Preregistration Template: https://osf.io/jea94/
- Center for Open Science YouTube: https://www.youtube.com/c/CenterforOpenScience

### Tools
- Power analysis: G*Power, pwr package (R), statsmodels (Python)
- Preregistration templates: OSF has 20+ templates for different study types
- Data repositories: Zenodo, Figshare, Dryad, Dataverse

## Next Steps

In **Module 10: Advanced Reproducibility & Containerization**, you'll learn:
- Computational reproducibility with Docker and containers
- Version control for data and models (DVC, Git-LFS)
- Workflow automation with Snakemake and Make
- Dependency management (conda, pip, renv)
- Reproducible random number generation
- Creating reproducible computational environments

Continue your journey toward fully transparent and reproducible research! 🚀